In [ ]:
%run ../python_classes/Emulator.ipynb

# Emulator class

Here we inspect methods of the Emulator class.

Create an emulator. For this example, we will just view July 1st at midnight.

In [ ]:
a = Emulator(
)

myFeaturesNames = ['acure_sea_spray', 'acure_dry_dep_acc', 'acure_bvoc_soa']

In [ ]:
a = Emulator(
    region=[-1, 0, -1, 0],
    excludeFeats=list(set(a.__get_input_names__()) - set(myFeaturesNames)),
    start_time="2017-07-01T10:00:00",
    end_time="2017-07-01T13:59:59",
    centerTarget=True
)

In [ ]:
a.__dict__

# Pixelwise

## Train

Train a model for each pixel.

In [ ]:
job = a.train()

In [ ]:
job.status()

In [ ]:
a.__dict__

Collect the parameters from each of the models.

In [ ]:
params = a.__retrieve_emulator_pixelwise__()

In [ ]:
params.status()

Display the learned parameters, excluding duplicates if necessary.

In [ ]:
df = a.__retrieve_pixelwise_parameters__(params)
df

## Emulate

In [ ]:
emu = a.__emulate_pixelwise__(nVariants=50)

In [ ]:
emu.status()

In [ ]:
a.excludeFeats

In [ ]:
a.__make_filter__()

In [ ]:
a.targetName

In [ ]:
a.__make_synth_dataset__(50)

In [ ]:
my_GPR_technique = c3.GaussianProcessRegressionTechnique(
    randomState=42,
    kernel=a.GPR_kernel,
    centerTarget=True
).upsert()

In [ ]:
my_GPR_technique.centerTarget=True

In [ ]:
my_GPR_technique

In [ ]:
job = c3.PredictAODGPR.makePredictionsJob(
            a.excludeFeats,
            a.__make_filter__(),
            a.targetName,
            a.__make_synth_dataset__(50), ###
            my_GPR_technique,
            a.batchSize)

In [ ]:
job = c3.AODGPRModelFinder.extractLearnedParametersJob(
            a.excludeFeats,
            a.__make_filter__(),
            a.targetName,
            #a.__make_synth_dataset__(50), ###
            my_GPR_technique,
            a.batchSize)

In [ ]:
job.status()

## Fit manually

In [ ]:
y_train = get_average_response(
    time1="2017-07-01T00:00:00",
    time2="2017-07-01T02:59:59",
    lat1 = -1.0,
    lat2 = 0.0,
    lon1 = -22.0,
    lon2 = -21.0
)

X_train = get_inputs()

In [ ]:
# create kernel
GPR_kernel = c3.SklearnGPRKernelMatern(lengthScale=[1.0]*len(myFeaturesNames), nu=0.5, coefficient=1.0).build().kernel.upsert()

# define technique
GPR_technique = c3.GaussianProcessRegressionTechnique(
    randomState=42,
    kernel=GPR_kernel,
    centerTarget=True
).upsert()

In [ ]:
# create pipe
GPR_pipe = c3.GaussianProcessRegressionPipe(
    technique=GPR_technique
)

In [ ]:
GPR_trained = GPR_pipe.train(
    input=c3.Dataset.fromPython(X_train),
    targetOutput=c3.Dataset.fromPython(pd.DataFrame(y_train))
)

In [ ]:
x = dict(zip(
    myFeaturesNames,
    c3.PythonSerialization.deserialize(serialized=GPR_trained.trainedModel.model).kernel_.get_params()['k2__length_scale']
))

dict(sorted(x.items(), key=lambda item: item[1]))

In [ ]:
df2.iloc[0, :]

## Get parameters manually

In [ ]:
# Filter data for job
gstpFilter = a2.make_filter()

In [ ]:
GPR_kernel = c3.SklearnGPRKernelMatern(
    lengthScale=[1.0]*len(a2.inputs),
    nu=0.5,
    coefficient=1.0
).build().kernel

In [ ]:
GPR_technique = c3.GaussianProcessRegressionTechnique(
    randomState = 42,
    kernel = GPR_kernel,
    centerTarget = a2.centerTarget
)

new_job = c3.AODGPRModelFinder.extractLearnedParametersJob(a2.excludeFeats, gstpFilter, "all", GPR_technique, 10)

In [ ]:
new_job.status()

In [ ]:
df = c3.AODGPRModelFinder.getDataframeFromJob(new_job)
df[["latitude", "longitude"]] = df[["latitude", "longitude"]].applymap(lambda x:float(x))
df["time"] = pd.DataFrame(df["time"]).applymap(lambda x:pd.Timestamp(str(x)))

In [ ]:
df.drop_duplicates(
    subset=["latitude", "longitude", "time"]
).reset_index(
    drop=True
).rename(
    columns=dict(zip(df.columns, a.inputs))
)

## In development: Regional emulator

In [ ]:
emulator = a.train(pixelwise=False, featureNames=a.inputs[53:], lengthscales=df, labels=[1]*df.shape[0])

In [ ]:
df_c = a.emulate_variant(pipe=emulator, queryVariant=np.random.rand(1, len(a.regionalFeatureNames))[0], lengthScales=df)

In [ ]:
df_c

In [ ]:
a.plot_variant(df_c)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
with open("lengthscale_tables/2017-07-01T00:00:00_2017-07-07T23:59:59", "r") as f:
    df = pd.read_csv(f)

In [ ]:
df

In [ ]:
model_ids = list(df.loc[0:10, "modelId"])

In [ ]:
c3.StagedFeatures.stageFromAODGPRModelIdsList(model_ids)

In [ ]:
c3.StagedTargets.stageFromAODGPRModelIdsList(model_ids)

In [ ]:
# create a pipe
kernelLen = 19
GPR_kernel = c3.SklearnGPRKernelMatern(lengthScale=[1.0]*kernelLen, nu=0.5, coefficient=1.0).build().kernel.upsert()

GPR_technique = c3.GaussianProcessRegressionTechnique(
    randomState=42,
    kernel=GPR_kernel,
    centerTarget=True
).upsert()

GPR_dataspec = c3.GPRDataSourceSpec(
    featuresType = c3.TypeRef(
        typeName="StagedFeatures"
    ),
    targetType=c3.TypeRef(
        typeName="StagedTargets"
    )
).upsert()

GPR_pipe = c3.GaussianProcessRegressionPipe(
    technique=GPR_technique,
    dataSourceSpec=GPR_dataspec
)

In [ ]:
# check features
features = GPR_pipe.getFeatures()
df_feats = c3.Dataset.toPandas(features)
df_feats

In [ ]:
# check targets
targets = GPR_pipe.getTarget()
df_targets = c3.Dataset.toPandas(targets)
df_targets

In [ ]:
GPR_trained = GPR_pipe.train(input=features, targetOutput=targets)

In [ ]:
dict(zip(df_feats, c3.PythonSerialization.deserialize(serialized=GPR_trained.trainedModel.model).kernel_.get_params()['k2__length_scale']))

In [ ]:
lengthScales = df.iloc[0:3, :]
labels = [0, 1, 2]

In [ ]:
regem = a2.train(pixelwise=False, lengthScales=lengthScales, labels=labels)

In [ ]:
regem

In [ ]:
queryVariant = pd.DataFrame(list(np.random.uniform(0, 1, 17)) + [-0.625, -0.9375]).transpose()
synth = c3.Dataset.fromPython(pythonData=queryVariant)

In [ ]:
for em in regem:
    pipe = c3.GaussianProcessRegressionPipe.get(regem[em].id)
    y_c = pipe.process(input=synth)
    print(c3.Dataset.toPandas(y_c))
    
    model = c3.PythonSerialization.deserialize(serialized=pipe.trainedModel.model)
    print(model.kernel_)